In [76]:
!pip install torch torchvision pytorch-transformers
!pip install scikit-learn pandas tensorflow

from __future__ import absolute_import, division, print_function
import torch

torch.manual_seed(0)
import csv
import logging
import os
import sys
from io import open

from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score

from multiprocessing import Pool, cpu_count
from tqdm import tqdm

logger = logging.getLogger(__name__)
csv.field_size_limit(2147483647)
# writerobj = SummaryWriter()

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir,data_type="dev"):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, data_type+".tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0","1","2","3"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            
            guid = "%s-%s" % (set_type, i)
            try:
                text_a = line[3]
                label = line[1]
            except:
                print(line)
            #print(text_a,label)
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


def convert_example_to_feature(example_row, pad_token=0,
sequence_a_segment_id=0, sequence_b_segment_id=1,
cls_token_segment_id=1, pad_token_segment_id=0,
mask_padding_with_zero=True):
    
   
    
    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id = example_row
    #print(example.text_a,"label-------->",example.label)
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    
    #print(label_map)

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)
    

def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True,
                                 process_count=cpu_count() - 2):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}
    #print(label_map)

    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id) for example in examples]

    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=100), total=len(examples)))

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

GLUE_TASKS_NUM_LABELS = {
    "binary": 4
}


# New Section

In [77]:
import pandas as pd
from tqdm import tqdm_notebook
import io
from google.colab import files
from sklearn.model_selection import train_test_split

# trainfile = files.upload()
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_df = pd.read_csv("/content/drive/My Drive/exp1/train2.tsv",delimiter="\t")
test_df = pd.read_csv("/content/drive/My Drive/exp1/test2.tsv",delimiter="\t")
dev_df = test_df

In [79]:
train_df

,text,label
0,I next attempted to make the agent more genera...,1
1,"The ability to recognize shapes, and detect th...",2
2,This agent still can’t solve problems which re...,2
3,"For Basic Problem D-06, not only does the shap...",1
4,"In that sense, for buildingunique solution to ...",1
...,...,...
271,Analysis of Generalization: The rules for CP’s...,2
272,"C-06, D-05 uses C-09, Challenge D-01 uses C-12...",1
273,For CP-3 again there is increasing dark pixels...,4
274,For both problems the XOR constraints hold ver...,4


In [80]:
train_df = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df["label"].astype(int)-1,
    'alpha':['a']*train_df.shape[0],
    'text': train_df["text"].replace(r'\n', ' ', regex=True)
})

train_df.head()

,id,label,alpha,text
0,0,0,a,I next attempted to make the agent more genera...
1,1,1,a,"The ability to recognize shapes, and detect th..."
2,2,1,a,This agent still can’t solve problems which re...
3,3,0,a,"For Basic Problem D-06, not only does the shap..."
4,4,0,a,"In that sense, for buildingunique solution to ..."


In [81]:
dev_df = pd.DataFrame({
    'id':range(len(dev_df)),
    'label':dev_df["label"].astype(int)-1,
    'alpha':['a']*dev_df.shape[0],
    'text': dev_df["text"].replace(r'\n', ' ', regex=True)
})

dev_df.head()

,id,label,alpha,text
0,0,1,a,The performance of the agent has increased a l...
1,1,1,a,This agent still can’t handle problems involvi...
2,2,1,a,This agent revision has improved performance a...
3,3,1,a,The Dark Pixel Ratio methodmust be fine-tuned ...
4,4,0,a,"Other than that, the root mean square differen..."


In [82]:
test_df = pd.DataFrame({
    'id':range(len(test_df)),
     'label':test_df["label"].astype(int)-1,
    'alpha':['a']*test_df.shape[0],
    'text': test_df["text"].replace(r'\n', ' ', regex=True)
})

test_df.head()

,id,label,alpha,text
0,0,1,a,The performance of the agent has increased a l...
1,1,1,a,This agent still can’t handle problems involvi...
2,2,1,a,This agent revision has improved performance a...
3,3,1,a,The Dark Pixel Ratio methodmust be fine-tuned ...
4,4,0,a,"Other than that, the root mean square differen..."


In [83]:
!mkdir data_new_1
train_df.to_csv('data_new_1/train.tsv', sep='\t', index=False, header=False, columns=train_df.columns)
dev_df.to_csv('data_new_1/dev.tsv', sep='\t', index=False, header=False, columns=dev_df.columns)
test_df.to_csv('data_new_1/test.tsv', sep='\t', index=False, header=False, columns=test_df.columns)

mkdir: cannot create directory ‘data_new_1’: File exists


In [84]:
!pip install pytorch_transformers


In [0]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

# from utils import (convert_examples_to_features,
#                         output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
args_bert = {
    'data_dir': 'data_new_1/',
    'model_type':  'bert',
    'model_name': 'bert-base-uncased',
    'task_name': 'binary',
    'output_dir': "outputs_bert-base-uncased_0.7_taskekant",
    'cache_dir': 'cache/',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 8,
    'eval_batch_size': 1,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 100,
    'weight_decay': 0,
    'learning_rate': 1e-10,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': False,
    'save_steps': 5000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': False,
    'reprocess_input_data': False,
    'notes': 'Using Yelp Reviews dataset'
}

args_roberta = {
    'data_dir': 'data_new_1/',
    'model_type':  'roberta',
    'model_name': 'roberta-base',
    'task_name': 'binary',
    'output_dir': 'outputs_robert-base-0.9_taskbc',
    'cache_dir': 'cache_roberta/',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 16,
    'eval_batch_size': 1,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 10,
    'weight_decay': 0,
    'learning_rate': 4e-8,
    'adam_epsilon': 1e-8,
    'warmup_steps': 1000,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': False,
    'save_steps': 5000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': False,
    'reprocess_input_data': False,
    'notes': 'Using Yelp Reviews dataset'
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
with open('args_bert.json', 'w') as f:
    json.dump(args_bert, f)
with open('args_roberta.json', 'w') as f:
    json.dump(args_roberta, f)

In [88]:
if os.path.exists(args_bert['output_dir']) and os.listdir(args_bert['output_dir']) and args_bert['do_train'] and not args_bert['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args_bert['output_dir']))

#Create roberta directory
if os.path.exists(args_roberta['output_dir']) and os.listdir(args_roberta['output_dir']) and args_roberta['do_train'] and not args_roberta['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args_roberta['output_dir']))

ValueError: ignored

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class_bert, model_class_bert, tokenizer_class_bert = MODEL_CLASSES[args_bert['model_type']]

MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

#roberta
config_class_roberta, model_class_roberta, tokenizer_class_roberta = MODEL_CLASSES[args_roberta['model_type']]


In [90]:
config_bert = config_class_bert.from_pretrained(args_bert['model_name'], num_labels=4, finetuning_task=args_bert['task_name'])#, output_hidden_states=True)
tokenizer_bert = tokenizer_class_bert.from_pretrained(args_bert['model_name'])



INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
model_bert = model_class_bert.from_pretrained(args_bert['model_name'],num_labels=4)
# for name, param in model_bert.named_parameters():
# 	if 'classifier' not in name: # classifier layer
# 		param.requires_grad = False

In [0]:

model_bert.to(device);


In [93]:
device

device(type='cuda', index=0)

In [0]:
task = args_bert['task_name']

processor = processors[task]()
label_list = processor.get_labels()
num_labels = len(label_list)

In [0]:
def load_and_cache_examples(task, tokenizer,argus, evaluate=False,undersample_scale_factor=1,data_type="dev"):
    args=argus.copy()
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(args['data_dir'], "cached_"+str(mode)+"_"+str(args['model_name'])+"_"+str(args['max_seq_length'])+"_"+str(task))
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data'] and False:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args['data_dir'],data_type) if evaluate else processor.get_train_examples(args['data_dir'])
#         print(len(examples))
        examples  = [example for example in examples]
        #print((examples))
        
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            process_count=2)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [0]:
def train(train_dataset, model, tokenizer,argus):
    global writerobj
    args=argus.copy()
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
    t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args['warmup_steps'], t_total=t_total)
    
    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args['fp16_opt_level'])
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
    
    for _ in train_iterator:
        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            #print(model.bert(**inputs).shape)
            loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)
            print("\r%f" % loss, end='')
            #writerobj.add_scalar('Loss/train', float(loss), global_step)
            

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']
                
               
                
 
            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])
                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])
            
            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
                
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                    # Log metrics
                    if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(model, tokenizer)
                        

                    logging_loss = tr_loss
                    

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], 'checkpoint')
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)
            

    return global_step, tr_loss / global_step

In [0]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix, classification_report
from scipy.stats import pearsonr

def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(args_bert['data_dir'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    print(classification_report(labels, preds))
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    print(classification_report(labels, preds))
   
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def evaluate(model, argus, tokenizer, prefix="",datatype="dev"):
    
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    args=argus.copy()
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, args, undersample_scale_factor=1, evaluate=True,data_type=datatype)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    #print(len(eval_dataset))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    probs=None
    net_input=None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            feature=model.bert(inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])[1].cpu().numpy()
            print(feature.shape)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        if preds is None:
            preds = logits.detach().cpu().numpy()
            probs= preds.copy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
            net_input=inputs['input_ids'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
            probs=np.append(probs,logits.detach().cpu().numpy(), axis=0)
            net_input=np.append(net_input,inputs['input_ids'].detach().cpu().numpy(),axis=0)
            
    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids)
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results, wrong,probs,preds,out_label_ids,net_input


def extract_features(model, argus, tokenizer, prefix="",datatype="dev"):
    
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    args=argus.copy()
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, args, undersample_scale_factor=1, evaluate=True,data_type=datatype)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    #print(len(eval_dataset))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])


    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    probs=None
    net_input=None
    
    #save data
    all_features=[]
    all_sentences=[]
    all_labels=[]

    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        # print(tokenizer.decode(batch[0])#.decode)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            feature=model.bert(inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])[1].cpu().numpy()
            
            all_features.append(feature[0])
            all_sentences.append(feature)
            all_labels.append(batch[3].cpu().numpy())

    return all_features,all_sentences,all_labels

In [0]:
#Train_bert
if args_bert['do_train'] and False:
    print("Training Bert!!")
    train_dataset = load_and_cache_examples(task, tokenizer_bert,args_bert,undersample_scale_factor=1)
    #print(train_dataset)
    global_step, tr_loss = train(train_dataset, model_bert, tokenizer_bert,args_bert)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

In [99]:
#Save bert
if args_bert['do_train']:
    if not os.path.exists(args_bert['output_dir']):
            os.makedirs(args_bert['output_dir'])
    logger.info("Saving model checkpoint to %s", args_bert['output_dir'])
    
    model_to_save = model_bert.module if hasattr(model_bert, 'module') else model_bert  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args_bert['output_dir'])
    tokenizer_bert.save_pretrained(args_bert['output_dir'])
    torch.save(args_bert, os.path.join(args_bert['output_dir'], 'training_args.bin'))


INFO:__main__:Saving model checkpoint to outputs_bert-base-uncased_0.7_taskekant


# Get test set predictions and probs for BERT

In [100]:
results = {}
if args_bert['do_eval']:
    checkpoints = [args_bert['output_dir']]
    if args_bert['eval_all_checkpoints']:
        checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args_bert['output_dir']+ '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        print(checkpoint)
        if True:
            global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
            model = model_class_bert.from_pretrained(checkpoint)
            model.to(device)
            xtest, sentence,ytest= extract_features(model, args_bert,tokenizer_bert, prefix=global_step,datatype="test")


results = {}
if args_bert['do_eval']:
    checkpoints = [args_bert['output_dir']]
    if args_bert['eval_all_checkpoints']:
        checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args_bert['output_dir']+ '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        print(checkpoint)
        if True:
            global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
            model = model_class_bert.from_pretrained(checkpoint)
            model.to(device)
            xtrain, sentence,ytrain= extract_features(model, args_bert,tokenizer_bert, prefix=global_step,datatype="train")


INFO:__main__:Evaluate the following checkpoints: ['outputs_bert-base-uncased_0.7_taskekant']


outputs_bert-base-uncased_0.7_taskekant


INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 184/184 [00:00<00:00, 1308.23it/s]
INFO:__main__:Saving features into cached file data_new_1/cached_dev_bert-base-uncased_128_binary
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 184
INFO:__main__:  Batch size = 1


INFO:__main__:Evaluate the following checkpoints: ['outputs_bert-base-uncased_0.7_taskekant']



outputs_bert-base-uncased_0.7_taskekant


INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 276/276 [00:00<00:00, 1233.72it/s]
INFO:__main__:Saving features into cached file data_new_1/cached_dev_bert-base-uncased_128_binary
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 276
INFO:__main__:  Batch size = 1


In [101]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from sklearn.dummy import DummyClassifier


params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

ytrain=np.array(ytrain).ravel()
ytest=np.array(ytest).ravel()


class_weights = list(class_weight.compute_class_weight('balanced',np.unique(ytrain),np.array(ytrain)))
w_array = np.ones(ytrain.shape[0], dtype = 'float')

for i, val in enumerate(ytrain):
    w_array[i] = class_weights[val-1]

clf= SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=100, random_state=42,class_weight="balanced",warm_start=True)
clf2=XGBClassifier(n_estimators=200,random_state=10,max_depth=3,learning_rate =0.1)

clf.fit(np.array(xtrain),np.array(ytrain).astype(np.int32))
clf2.fit(np.array(xtrain),np.array(ytrain).astype(np.int32))

ypred=clf.predict(xtest)
ypred2=clf2.predict(xtest)

from sklearn.metrics import classification_report

print("SGD")
print(classification_report(np.array(ytest).ravel(),ypred,digits=5))


print("XGB")
print(classification_report(np.array(ytest).ravel(),ypred2,digits=5))

# files = open('xtrain.pkl', 'wb')

# # dump information to that file
# pickle.dump(xtrain, files)
# files = open('ytrain.pkl', 'wb')

# # dump information to that file
# pickle.dump(ytrain, files)

# files = open('xtest.pkl', 'wb')

# # dump information to that file
# pickle.dump(xtest, files)

# files = open('ytest.pkl', 'wb')

# # dump information to that file
# pickle.dump(ytest, files)
# files.close()


SGD
              precision    recall  f1-score   support

           0    0.76000   0.33929   0.46914        56
           1    0.57778   0.86667   0.69333        90
           2    0.30769   0.26667   0.28571        15
           3    0.27273   0.13043   0.17647        23

    accuracy                        0.56522       184
   macro avg    0.47955   0.40076   0.40616       184
weighted avg    0.57309   0.56522   0.52726       184

XGB
              precision    recall  f1-score   support

           0    0.55263   0.37500   0.44681        56
           1    0.59690   0.85556   0.70320        90
           2    0.50000   0.06667   0.11765        15
           3    0.33333   0.21739   0.26316        23

    accuracy                        0.56522       184
   macro avg    0.49572   0.37865   0.38270       184
weighted avg    0.54258   0.56522   0.52243       184



In [0]:
!pip install eli5

In [0]:
!mkdir Fold2

# BERT INTERPRETABILITY

In [0]:
import pickle
import eli5
from eli5.lime import TextExplainer

xtrain=pickle.load(open('./xtrain.pkl', 'rb'))
xtest=pickle.load(open('./xtest.pkl', 'rb'))

ytrain=pickle.load(open('./ytrain.pkl', 'rb'))
ytest=pickle.load(open('./ytest.pkl', 'rb'))




traindata=pd.read_csv("./test2.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("./train2.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)


xtraindict={}


for i,j in zip(X_train,xtrain):
    xtraindict[i]=j

for i,j in zip(X_test,xtest):
    xtraindict[i]=j

def returnfeatures(text):
    global xtraindict
    res=[]
    for i in text:
        res.append(xtraindict[i])
    return np.array(res)

clf11 = Pipeline([('custom', FunctionTransformer(returnfeatures)),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=200, random_state=42,class_weight="balanced",warm_start=True))])

clf11.fit(X_train,ytrain)
ypred=clf11.predict(X_test)

print(classification_report(ypred,ytest,digits=5))


clf11 = Pipeline([('custom', FunctionTransformer(returnfeatures)),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=200, random_state=42,class_weight="balanced",warm_start=True))])

clf11.fit(X_train,ytrain)
ypred=clf11.predict(X_test)
print(classification_report(ypred,ytest,digits=5))


clf12=XGBClassifier(n_estimators=200,random_state=10,max_depth=3,learning_rate =0.1)
clf11.fit(X_train,ytrain)
ypred=clf11.predict(X_test)
print(classification_report(ypred,ytest,digits=5))

